# DATA CLEANING AND PREPROCESSING

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,OrdinalEncoder,StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from datetime import datetime
import urllib.request
from bs4 import BeautifulSoup
import json
from urllib.request import Request, urlopen
import numpy as np
import pandas as pd
import ssl
import requests
from statistics import mean
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
input_file = 'plans_v3.2.xlsx'

In [3]:
to_process = ["Commercial", "Medicare", "State_medicaid", "Health_Exchange", "Managed medicaid"]

In [4]:
def melt(df):
    df.reset_index(inplace=True)
    return pd.melt(df,id_vars=['index'])

In [5]:
def clean_percentage_data(df_status):
    df_status = df_status.astype(str)
    for col in df_status.columns:
        if "unnamed" in col.lower():
            df_status = df_status.drop(col, axis=1)

    df_p = df_status.drop("# of Plans", axis=1)

    # Dropping unnecessary columns and rows and creating separate dataframes for analysis
    df_p = df_p.iloc[:-2,:]
    truncate_idx = df_p.shape[0]
    for col in df_p.columns[1:]:
        for idx, i in enumerate(range(len(df_p[col].values))):
            parts = str(df_p[col].values[i]).split('%')
            if len(parts)==1:
                truncate_idx = min(idx,truncate_idx)
            df_p[col].values[i] = parts[0]

    df_p = df_p.iloc[:truncate_idx]

    df_p.rename(columns = {'Payer/PBM':'Trade Name'},inplace=True)
    df_pt = df_p.transpose()
    new_header = df_pt.iloc[0]
    df_pt = df_pt[1:] 
    df_pt.columns = new_header
    return df_pt.astype(float)

In [6]:
def clean_coverage_data(df_status):
    df_status = df_status.astype(str)
    for col in df_status.columns:
        if "unnamed" in col.lower():
            df_status = df_status.drop(col, axis=1)

    df_c = df_status.drop("# of Plans", axis=1)
    df_c = df_c.iloc[:-2,:]
    #truncate_idx = df_p.shape[0]
    for col in df_c.columns[1:]:
        for idx, i in enumerate(range(len(df_c[col].values))):
            status = ""
            if "not covered" in str(df_c[col].values[i]).lower():
                status = "Not Covered"
            elif "covered (pa/st)" in str(df_c[col].values[i]).lower():
                status = "Covered (PA/ST)"
            elif "covered" in str(df_c[col].values[i]).lower():
                status = "Covered"
            elif "preferred (pa/st)" in str(df_c[col].values[i]).lower():
                status = "Preferred (PA/ST)"
            elif "preferred" in str(df_c[col].values[i]).lower():
                status = "Preferred"
            else:
                status = "Other"
            df_c[col].values[i] = status

    df_c.rename(columns = {'Payer/PBM':'Trade Name'},inplace=True)
    df_ct = df_c.transpose()
    new_header = df_ct.iloc[0]
    df_ct = df_ct[1:] 
    df_ct.columns = new_header 
    return df_ct

In [7]:
def get_plans(df_status):
    #df_status = df_status.astype(str)
    for col in df_status.columns:
        if "unnamed" in col.lower():
            df_status = df_status.drop(col, axis=1)
    df_status = df_status.iloc[:-2,:]
    df_status = df_status[['Payer/PBM','# of Plans']]
    df_status['# of Plans'] = df_status['# of Plans'].str.split(' ').str[0]
    df_status['# of Plans'] = df_status['# of Plans'].astype('int64')
    return df_status

## Commercial

In [8]:
sheet_name = to_process[0]
print (sheet_name)
df = pd.read_excel(input_file,sheet_name = sheet_name)
df.head(10)

Commercial


,Payer/PBM,# of Plans,Vijoice,Fyarro,Carvykti,Tecartus,Enhertu,Exkivity,Pluvicto,Cabometyx,...,Ayvakit,Tukysa,Qinlock,Bavencio,Blincyto,Cosela,Jelmyto,Polivy,Rozlytrek,Turalio
0,CVS Health (Aetna),1217 plans,72% Not Covered,96% Not Covered,100% Not Covered,85% Not Covered,75% Covered (PA/ST),85% Covered (PA/ST),100% Not Covered,69% Preferred (PA/ST),...,73% Covered (PA/ST),81% Covered (PA/ST),71% Covered (PA/ST),68% Covered (PA/ST),79% Covered (PA/ST),65% Not Covered,89% Not Covered,68% Covered (PA/ST),68% Preferred (PA/ST),68% Covered (PA/ST)
1,Express Scripts PBM,1179 plans,96% Not Covered,93% Covered (PA/ST),91% Covered (PA/ST),91% Covered (PA/ST),50% Covered,91% Covered (PA/ST),97% Not Covered,68% Preferred (PA/ST),...,97% Covered (PA/ST),94% Covered (PA/ST),92% Not Covered,66% Preferred (PA/ST),67% Preferred (PA/ST),89% Not Covered,92% Covered (PA/ST),94% Covered,92% Preferred (PA/ST),91% Covered
2,"UnitedHealth Group, Inc.",119 plans,97% Covered (PA/ST),100% Not Covered,100% Not Covered,99% Covered,99% Covered,100% Covered (PA/ST),100% Not Covered,82% Covered (PA/ST),...,99% Covered (PA/ST),89% Covered (PA/ST),97% Covered (PA/ST),100% Covered,99% Covered,100% Covered,100% Covered,100% Covered,91% Preferred (PA/ST),90% Covered
3,Cigna Corporation,68 plans,100% Not Covered,95% Covered (PA/ST),100% Not Covered,95% Not Covered,97% Covered (PA/ST),97% Covered (PA/ST),100% Not Covered,59% Covered (PA/ST),...,100% Covered (PA/ST),100% Covered (PA/ST),97% Covered (PA/ST),77% Not Covered,77% Not Covered,95% Not Covered,97% Covered (PA/ST),77% Not Covered,97% Covered (PA/ST),95% Covered (PA/ST)
4,OptumRx,206 plans,66% Covered (PA/ST),78% Covered (PA/ST),66% Covered (PA/ST),69% Covered (PA/ST),78% Covered (PA/ST),80% Covered (PA/ST),97% Not Covered,95% Preferred (PA/ST),...,80% Covered (PA/ST),100% Covered (PA/ST),82% Covered (PA/ST),78% Covered (PA/ST),78% Covered (PA/ST),80% Covered,79% Covered,78% Covered (PA/ST),100% Covered (PA/ST),80% Covered (PA/ST)
5,Department of Defense - TRICARE,5 plans,100% Covered (PA/ST),100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Preferred (PA/ST),100% Covered,100% Preferred,...,100% Preferred (PA/ST),100% Preferred (PA/ST),100% Preferred (PA/ST),100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Preferred (PA/ST),100% Preferred (PA/ST)
6,"Kaiser Foundation Health Plans, Inc.",41 plans,71% Covered,71% Covered,71% Covered,72% Covered,71% Preferred,72% Covered,100% Not Covered,58% Preferred,...,80% Not Covered,54% Preferred,72% Covered,73% Covered,48% Covered,72% Covered,84% Not Covered,72% Covered,54% Preferred,72% Covered
7,"Anthem, Inc.",198 plans,100% Covered (PA/ST),98% Covered (PA/ST),98% Not Covered,98% Not Covered,100% Covered (PA/ST),100% Covered (PA/ST),98% Covered (PA/ST),98% Preferred (PA/ST),...,100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),98% Covered (PA/ST),98% Covered (PA/ST),98% Covered (PA/ST)
8,Blue Cross Blue Shield Association Corporation,6 plans,100% Covered (PA/ST),100% Not Covered,100% Not Covered,99% Covered,100% Covered (PA/ST),100% Covered (PA/ST),100% Not Covered,99% Covered (PA/ST),...,100% Covered (PA/ST),99% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),99% Covered (PA/ST),99% Covered (PA/ST),99% Covered,100% Covered (PA/ST),99% Covered (PA/ST),99% Covered (PA/ST)
9,Health Care Service Corporation,115 plans,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,51% Covered (PA/ST),100% Not Covered,100% Covered (PA/ST),...,99% Covered (PA/ST),95% Covered (PA/ST),99% Covered (PA/ST),51% Not Covered,94% Not Covered,51% Not Covered,88% Not Covered,100% Not Covered,98% Covered (PA/ST),84% Covered (PA/ST)


In [9]:
df_commercial_p = clean_percentage_data(df) # Commercial
df_commercial_p

Trade Name,CVS Health (Aetna),Express Scripts PBM,"UnitedHealth Group, Inc.",Cigna Corporation,OptumRx,Department of Defense - TRICARE,"Kaiser Foundation Health Plans, Inc.","Anthem, Inc.",Blue Cross Blue Shield Association Corporation,Health Care Service Corporation,...,Alameda Alliance for Health,"GlobalHealth, Inc.",Chinese Community Health Plan Corporation,Washtenaw Health Plan,Samaritan Health Services,Santa Cruz-Monterey Managed Medical Care Commission,ClearScript PBM,University Health System Inc.,Global Pharmaceutical Benefits,Centers for Disease Control and Prevention
Vijoice,72.0,96.0,97.0,100.0,66.0,100.0,71.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
Fyarro,96.0,93.0,100.0,95.0,78.0,100.0,71.0,98.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
Carvykti,100.0,91.0,100.0,100.0,66.0,100.0,71.0,98.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
Tecartus,85.0,91.0,99.0,95.0,69.0,100.0,72.0,98.0,99.0,100.0,...,100.0,100.0,100.0,100.0,60.0,100.0,100.0,100.0,100.0,100.0
Enhertu,75.0,50.0,99.0,97.0,78.0,100.0,71.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Cosela,65.0,89.0,100.0,95.0,80.0,100.0,72.0,100.0,99.0,51.0,...,100.0,100.0,100.0,100.0,60.0,100.0,100.0,100.0,100.0,100.0
Jelmyto,89.0,92.0,100.0,97.0,79.0,100.0,84.0,100.0,99.0,88.0,...,100.0,100.0,100.0,100.0,60.0,100.0,100.0,100.0,100.0,100.0
Polivy,68.0,94.0,100.0,77.0,78.0,100.0,72.0,98.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
Rozlytrek,68.0,92.0,91.0,97.0,100.0,100.0,54.0,98.0,99.0,98.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0


In [10]:
df_commercial_c = clean_coverage_data(df) # Commercial
df_commercial_c

Trade Name,CVS Health (Aetna),Express Scripts PBM,"UnitedHealth Group, Inc.",Cigna Corporation,OptumRx,Department of Defense - TRICARE,"Kaiser Foundation Health Plans, Inc.","Anthem, Inc.",Blue Cross Blue Shield Association Corporation,Health Care Service Corporation,...,MedalistRx PBM,Citizens Rx PBM,PharmPix,TrueScripts PBM,Pharmavail PBM,Araya PBM,Abarca Health,ProAct PBM,Prescryptive PBM,Rightway PBM
Vijoice,Not Covered,Not Covered,Covered (PA/ST),Not Covered,Covered (PA/ST),Covered (PA/ST),Covered,Covered (PA/ST),Covered (PA/ST),Not Covered,...,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Covered (PA/ST)
Fyarro,Not Covered,Covered (PA/ST),Not Covered,Covered (PA/ST),Covered (PA/ST),Not Covered,Covered,Covered (PA/ST),Not Covered,Not Covered,...,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered
Carvykti,Not Covered,Covered (PA/ST),Not Covered,Not Covered,Covered (PA/ST),Not Covered,Covered,Not Covered,Not Covered,Not Covered,...,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered
Tecartus,Not Covered,Covered (PA/ST),Covered,Not Covered,Covered (PA/ST),Not Covered,Covered,Not Covered,Covered,Not Covered,...,Covered,Covered,Covered,Covered,Covered,Covered,Covered,Covered,Covered,Covered
Enhertu,Covered (PA/ST),Covered,Covered,Covered (PA/ST),Covered (PA/ST),Not Covered,Preferred,Covered (PA/ST),Covered (PA/ST),Not Covered,...,Covered,Covered,Covered,Covered,Covered,Covered,Covered,Covered,Covered,Covered
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Cosela,Not Covered,Not Covered,Covered,Not Covered,Covered,Not Covered,Covered,Covered (PA/ST),Covered (PA/ST),Not Covered,...,Covered,Covered,Covered,Covered,Covered,Covered,Covered,Covered,Covered,Covered
Jelmyto,Not Covered,Covered (PA/ST),Covered,Covered (PA/ST),Covered,Not Covered,Not Covered,Covered (PA/ST),Covered,Not Covered,...,Covered,Covered,Covered,Covered,Covered,Covered,Covered,Covered,Covered,Covered
Polivy,Covered (PA/ST),Covered,Covered,Not Covered,Covered (PA/ST),Not Covered,Covered,Covered (PA/ST),Covered (PA/ST),Not Covered,...,Not Covered,Covered,Covered,Covered,Covered,Covered,Covered,Covered,Covered,Covered
Rozlytrek,Preferred (PA/ST),Preferred (PA/ST),Preferred (PA/ST),Covered (PA/ST),Covered (PA/ST),Preferred (PA/ST),Preferred,Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),...,Not Covered,Not Covered,Covered,Covered,Covered,Covered,Covered,Covered,Covered,Covered


In [11]:
df_commercial_num = get_plans(df) # Commercial
df_commercial_num

,Payer/PBM,# of Plans
0,CVS Health (Aetna),1217
1,Express Scripts PBM,1179
2,"UnitedHealth Group, Inc.",119
3,Cigna Corporation,68
4,OptumRx,206
...,...,...
183,Araya PBM,2
184,Abarca Health,1
185,ProAct PBM,6
186,Prescryptive PBM,1


## Medicare

In [12]:
sheet_name = to_process[1]
print (sheet_name)
df = pd.read_excel(input_file,sheet_name = sheet_name)
df.head(10)

Medicare


,Payer/PBM,# of Plans,Vijoice,Fyarro,Carvykti,Tecartus,Enhertu,Exkivity,Pluvicto,Cabometyx,...,Tukysa,Qinlock,Bavencio,Blincyto,Cosela,Jelmyto,Polivy,Rozlytrek,Turalio,Pemazyre
0,"UnitedHealth Group, Inc.",132 plans,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,78% Covered (PA/ST),100% Not Covered,82% Covered (PA/ST),...,81% Covered,78% Covered (PA/ST),95% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,78% Covered (PA/ST),78% Covered (PA/ST),78% Covered (PA/ST)
1,CVS Health (Aetna),137 plans,90% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,62% Not Covered,75% Covered (PA/ST),100% Not Covered,99% Covered (PA/ST),...,100% Covered (PA/ST),93% Covered (PA/ST),90% Not Covered,74% Not Covered,100% Not Covered,100% Not Covered,62% Not Covered,100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST)
2,"Humana, Inc.",95 plans,93% Not Covered,100% Not Covered,93% Not Covered,100% Not Covered,58% Covered (PA/ST),100% Covered (PA/ST),100% Not Covered,100% Covered (PA/ST),...,100% Covered (PA/ST),100% Covered (PA/ST),58% Covered (PA/ST),100% Not Covered,100% Not Covered,100% Not Covered,58% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST)
3,Centene Corporation,188 plans,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,99% Covered (PA/ST),100% Not Covered,99% Covered (PA/ST),...,99% Covered (PA/ST),99% Covered (PA/ST),100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,99% Covered (PA/ST),99% Covered (PA/ST),99% Covered (PA/ST)
4,"Anthem, Inc.",166 plans,86% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,86% Covered (PA/ST),93% Covered (PA/ST),100% Not Covered,93% Covered (PA/ST),...,93% Covered (PA/ST),93% Covered (PA/ST),86% Covered (PA/ST),71% Covered (PA/ST),100% Not Covered,100% Not Covered,86% Covered (PA/ST),93% Covered (PA/ST),93% Covered (PA/ST),93% Covered (PA/ST)
5,Cigna Corporation,28 plans,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,95% Covered (PA/ST),95% Covered (PA/ST),100% Not Covered,95% Covered (PA/ST),...,95% Covered (PA/ST),95% Covered (PA/ST),95% Covered (PA/ST),95% Covered,100% Not Covered,100% Not Covered,95% Covered (PA/ST),95% Covered (PA/ST),95% Covered (PA/ST),95% Covered (PA/ST)
6,"Kaiser Foundation Health Plans, Inc.",70 plans,95% Covered,95% Covered,100% Not Covered,100% Not Covered,95% Covered,95% Covered,100% Not Covered,89% Covered,...,95% Covered,95% Covered,95% Covered,89% Not Covered,89% Covered,100% Not Covered,95% Covered,95% Covered,95% Covered,95% Covered
7,Express Scripts PBM,4 plans,50% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,56% Covered (PA/ST),100% Not Covered,56% Covered (PA/ST),...,68% Covered (PA/ST),68% Covered (PA/ST),63% Not Covered,63% Not Covered,100% Not Covered,100% Not Covered,63% Not Covered,68% Covered (PA/ST),68% Covered (PA/ST),68% Covered (PA/ST)
8,Elixir Insurance,3 plans,100% Covered (PA/ST),100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Covered (PA/ST),100% Not Covered,100% Covered (PA/ST),...,100% Covered (PA/ST),100% Covered (PA/ST),100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST)
9,OptumRx,3 plans,59% Covered (PA/ST),59% Covered (PA/ST),100% Not Covered,100% Not Covered,100% Covered (PA/ST),59% Covered (PA/ST),100% Not Covered,100% Covered (PA/ST),...,100% Covered (PA/ST),100% Covered (PA/ST),41% Covered (PA/ST),76% Covered (PA/ST),100% Not Covered,100% Not Covered,100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST)


In [13]:
df_medicare_p = clean_percentage_data(df) # Medicare
df_medicare_p

Trade Name,"UnitedHealth Group, Inc.",CVS Health (Aetna),"Humana, Inc.",Centene Corporation,"Anthem, Inc.",Cigna Corporation,"Kaiser Foundation Health Plans, Inc.",Express Scripts PBM,Elixir Insurance,OptumRx,...,Aspirus Health Care,Vitality Health Plan of California,RiverSpring Health Plans,"Community Health Choice, Inc.",Dignity Health Plan,McLaren Health Care Corporation,"Nascentia Health, Inc.","Sonder Health Plans, Inc.","El Paso HealthPlans, Inc.",University Health System Inc.
Vijoice,100.0,90.0,93.0,100.0,86.0,100.0,95.0,50.0,100.0,59.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
Fyarro,100.0,100.0,100.0,100.0,100.0,100.0,95.0,100.0,100.0,59.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
Carvykti,100.0,100.0,93.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
Tecartus,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
Enhertu,100.0,62.0,58.0,100.0,86.0,95.0,95.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,54.0,100.0,100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Jelmyto,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
Polivy,100.0,62.0,58.0,100.0,86.0,95.0,95.0,63.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,54.0,100.0,100.0,100.0,100.0
Rozlytrek,78.0,100.0,100.0,99.0,93.0,95.0,95.0,68.0,100.0,100.0,...,100.0,100.0,61.0,100.0,100.0,54.0,100.0,100.0,100.0,100.0
Turalio,78.0,100.0,100.0,99.0,93.0,95.0,95.0,68.0,100.0,100.0,...,100.0,100.0,61.0,100.0,100.0,54.0,100.0,100.0,100.0,100.0


In [14]:
df_medicare_c = clean_coverage_data(df) # Medicare
df_medicare_c

Trade Name,"UnitedHealth Group, Inc.",CVS Health (Aetna),"Humana, Inc.",Centene Corporation,"Anthem, Inc.",Cigna Corporation,"Kaiser Foundation Health Plans, Inc.",Express Scripts PBM,Elixir Insurance,OptumRx,...,"El Paso HealthPlans, Inc.",University Health System Inc.,Magellan Rx Management,State of North Carolina,Golden State Medicare Health Plan,Integra Managed Care,Group Health Cooperative Eau Claire,eternalHealth,"Innovative Integrated Health, Inc.",ApexHealth
Vijoice,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Covered,Not Covered,Covered (PA/ST),Covered (PA/ST),...,Not Covered,Not Covered,Not Covered,Covered (PA/ST),Not Covered,Covered (PA/ST),Not Covered,Not Covered,Not Covered,Not Covered
Fyarro,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Covered,Not Covered,Not Covered,Covered (PA/ST),...,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered
Carvykti,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,...,Not Covered,Not Covered,Not Covered,Covered (PA/ST),Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered
Tecartus,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,...,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered
Enhertu,Not Covered,Not Covered,Covered (PA/ST),Not Covered,Covered (PA/ST),Covered (PA/ST),Covered,Not Covered,Not Covered,Covered (PA/ST),...,Not Covered,Not Covered,Not Covered,Covered (PA/ST),Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Covered (PA/ST)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Jelmyto,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,...,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Not Covered
Polivy,Not Covered,Not Covered,Covered (PA/ST),Not Covered,Covered (PA/ST),Covered (PA/ST),Covered,Not Covered,Not Covered,Covered (PA/ST),...,Not Covered,Not Covered,Not Covered,Covered (PA/ST),Not Covered,Not Covered,Not Covered,Covered (PA/ST),Not Covered,Covered (PA/ST)
Rozlytrek,Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered,Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),...,Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST)
Turalio,Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered,Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),...,Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Not Covered


In [15]:
df_medicare_num = get_plans(df) # Medicare
df_medicare_num

,Payer/PBM,# of Plans
0,"UnitedHealth Group, Inc.",132
1,CVS Health (Aetna),137
2,"Humana, Inc.",95
3,Centene Corporation,188
4,"Anthem, Inc.",166
...,...,...
207,Integra Managed Care,2
208,Group Health Cooperative Eau Claire,2
209,eternalHealth,3
210,"Innovative Integrated Health, Inc.",1


## State_medicaid

In [16]:
sheet_name = to_process[2]
print (sheet_name)
df = pd.read_excel(input_file,sheet_name = sheet_name)
df.head(10)

State_medicaid


,Payer/PBM,# of Plans,Vijoice,Fyarro,Carvykti,Tecartus,Enhertu,Exkivity,Pluvicto,Cabometyx,...,Ayvakit,Tukysa,Qinlock,Bavencio,Blincyto,Cosela,Jelmyto,Polivy,Rozlytrek,Turalio
0,State of Colorado,7 plans,100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered,100% Covered,100% Not Covered,100% Covered,...,100% Covered,100% Covered,100% Covered,100% Covered,100% Covered,100% Covered,100% Covered,100% Covered,100% Covered,100% Covered
1,State of Oklahoma,2 plans,100% Not Covered,100% Not Covered,100% Not Covered,100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),...,100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered,100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST)
2,State of Massachusetts,2 plans,100% Preferred (PA/ST),100% Covered (PA/ST),100% Not Covered,100% Not Covered,100% Covered (PA/ST),100% Covered (PA/ST),100% Not Covered,100% Covered (PA/ST),...,100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Not Covered,100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST)
3,State of California,2 plans,99% Covered (PA/ST),99% Preferred,99% Covered (PA/ST),99% Covered (PA/ST),99% Preferred,99% Preferred,100% Not Covered,99% Preferred,...,99% Preferred,99% Covered (PA/ST),99% Preferred,99% Preferred,99% Preferred,99% Covered (PA/ST),99% Preferred,99% Preferred,99% Preferred,99% Preferred
4,State of Florida,2 plans,98% Covered (PA/ST),98% Covered (PA/ST),98% Covered (PA/ST),98% Covered,98% Covered,98% Covered (PA/ST),100% Not Covered,98% Covered (PA/ST),...,98% Covered,98% Covered,98% Covered,98% Covered (PA/ST),98% Covered (PA/ST),98% Covered (PA/ST),98% Covered (PA/ST),98% Covered (PA/ST),98% Covered,98% Covered
5,State of Alabama,2 plans,100% Covered,100% Not Covered,100% Covered,100% Not Covered,100% Covered,100% Covered,100% Not Covered,100% Covered,...,100% Covered (PA/ST),100% Covered,100% Covered,100% Covered,100% Covered,100% Not Covered,100% Covered,100% Covered,100% Covered,100% Covered
6,State of Arkansas,2 plans,100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered,100% Covered,100% Covered (PA/ST),100% Not Covered,100% Covered (PA/ST),...,100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered,100% Covered,100% Covered,100% Covered,100% Covered,100% Covered (PA/ST),100% Covered (PA/ST)
7,State of Connecticut,2 plans,100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered,100% Covered,100% Preferred,100% Not Covered,100% Preferred,...,100% Preferred,100% Covered,100% Preferred,100% Covered,100% Covered,100% Covered,100% Covered,100% Covered,100% Preferred,100% Preferred
8,State of Michigan,2 plans,100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Not Covered,100% Not Covered,100% Covered,100% Not Covered,100% Covered,...,100% Covered,100% Covered,100% Covered,100% Not Covered,100% Not Covered,100% Covered,100% Not Covered,100% Covered,100% Covered,100% Covered
9,State of Georgia,2 plans,98% Covered (PA/ST),98% Covered (PA/ST),98% Covered (PA/ST),98% Covered,98% Covered,98% Preferred (PA/ST),100% Not Covered,98% Preferred (PA/ST),...,98% Preferred (PA/ST),98% Preferred (PA/ST),98% Preferred (PA/ST),98% Covered,98% Covered,98% Covered,98% Covered,98% Covered,98% Preferred (PA/ST),98% Preferred (PA/ST)


In [17]:
df_state_medicaid_p = clean_percentage_data(df) # State_medicaid
df_state_medicaid_p

Trade Name,State of Colorado,State of Oklahoma,State of Massachusetts,State of California,State of Florida,State of Alabama,State of Arkansas,State of Connecticut,State of Michigan,State of Georgia,...,State of Delaware,State of West Virginia,State of Rhode Island,Commonwealth of Puerto Rico,State of Tennessee,State of New Hampshire,State of Nebraska,District of Columbia,State of Kansas,State of Hawaii
Vijoice,100.0,100.0,100.0,99.0,98.0,100.0,100.0,100.0,100.0,98.0,...,100.0,99.0,98.0,100.0,100.0,91.0,67.0,100.0,100.0,100.0
Fyarro,100.0,100.0,100.0,99.0,98.0,100.0,100.0,100.0,100.0,98.0,...,100.0,99.0,98.0,100.0,100.0,91.0,100.0,100.0,100.0,100.0
Carvykti,100.0,100.0,100.0,99.0,98.0,100.0,100.0,100.0,100.0,98.0,...,100.0,99.0,98.0,100.0,100.0,91.0,100.0,100.0,100.0,100.0
Tecartus,100.0,100.0,100.0,99.0,98.0,100.0,100.0,100.0,100.0,98.0,...,100.0,99.0,98.0,100.0,100.0,91.0,67.0,100.0,100.0,100.0
Enhertu,100.0,100.0,100.0,99.0,98.0,100.0,100.0,100.0,100.0,98.0,...,100.0,99.0,98.0,100.0,100.0,91.0,67.0,100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Cosela,100.0,100.0,100.0,99.0,98.0,100.0,100.0,100.0,100.0,98.0,...,100.0,99.0,98.0,100.0,100.0,91.0,67.0,100.0,100.0,100.0
Jelmyto,100.0,100.0,100.0,99.0,98.0,100.0,100.0,100.0,100.0,98.0,...,100.0,99.0,98.0,100.0,100.0,91.0,67.0,100.0,100.0,100.0
Polivy,100.0,100.0,100.0,99.0,98.0,100.0,100.0,100.0,100.0,98.0,...,100.0,99.0,98.0,100.0,100.0,91.0,67.0,100.0,100.0,100.0
Rozlytrek,100.0,100.0,100.0,99.0,98.0,100.0,100.0,100.0,100.0,98.0,...,100.0,99.0,98.0,100.0,100.0,91.0,67.0,100.0,100.0,100.0


In [18]:
df_state_medicaid_c = clean_coverage_data(df) # State_medicaid
df_state_medicaid_c

Trade Name,State of Colorado,State of Oklahoma,State of Massachusetts,State of California,State of Florida,State of Alabama,State of Arkansas,State of Connecticut,State of Michigan,State of Georgia,...,State of Delaware,State of West Virginia,State of Rhode Island,Commonwealth of Puerto Rico,State of Tennessee,State of New Hampshire,State of Nebraska,District of Columbia,State of Kansas,State of Hawaii
Vijoice,Covered (PA/ST),Not Covered,Preferred (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered,Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),...,Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Not Covered,Not Covered,Covered (PA/ST),Covered,Not Covered,Not Covered,Not Covered
Fyarro,Covered (PA/ST),Not Covered,Covered (PA/ST),Preferred,Covered (PA/ST),Not Covered,Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),...,Not Covered,Covered (PA/ST),Covered (PA/ST),Not Covered,Not Covered,Covered,Not Covered,Not Covered,Not Covered,Not Covered
Carvykti,Covered (PA/ST),Not Covered,Not Covered,Covered (PA/ST),Covered (PA/ST),Covered,Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),...,Covered,Covered (PA/ST),Covered (PA/ST),Not Covered,Not Covered,Covered (PA/ST),Not Covered,Not Covered,Not Covered,Not Covered
Tecartus,Covered (PA/ST),Covered (PA/ST),Not Covered,Covered (PA/ST),Covered,Not Covered,Covered,Covered,Not Covered,Covered,...,Preferred (PA/ST),Covered,Covered,Not Covered,Not Covered,Covered,Covered,Not Covered,Not Covered,Not Covered
Enhertu,Covered,Covered (PA/ST),Covered (PA/ST),Preferred,Covered,Covered,Covered,Covered,Not Covered,Covered,...,Preferred (PA/ST),Covered,Covered,Not Covered,Not Covered,Covered,Covered,Not Covered,Not Covered,Not Covered
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Cosela,Covered,Covered,Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Not Covered,Covered,Covered,Covered,Covered,...,Covered,Covered,Covered,Not Covered,Not Covered,Covered,Covered,Not Covered,Not Covered,Not Covered
Jelmyto,Covered,Covered (PA/ST),Not Covered,Preferred,Covered (PA/ST),Covered,Covered,Covered,Not Covered,Covered,...,Not Covered,Covered,Covered,Not Covered,Not Covered,Covered,Covered,Not Covered,Not Covered,Not Covered
Polivy,Covered,Covered (PA/ST),Covered (PA/ST),Preferred,Covered (PA/ST),Covered,Covered,Covered,Covered,Covered,...,Preferred (PA/ST),Covered,Covered,Not Covered,Not Covered,Covered,Covered,Not Covered,Not Covered,Not Covered
Rozlytrek,Covered,Covered (PA/ST),Covered (PA/ST),Preferred,Covered,Covered,Covered (PA/ST),Preferred,Covered,Preferred (PA/ST),...,Preferred (PA/ST),Covered,Covered,Not Covered,Not Covered,Covered,Covered (PA/ST),Not Covered,Not Covered,Not Covered


In [19]:
df_state_medicaid_num = get_plans(df) # State_medicaid
df_state_medicaid_num

,Payer/PBM,# of Plans
0,State of Colorado,7
1,State of Oklahoma,2
2,State of Massachusetts,2
3,State of California,2
4,State of Florida,2
5,State of Alabama,2
6,State of Arkansas,2
7,State of Connecticut,2
8,State of Michigan,2
9,State of Georgia,2


## Health_Exchange

In [20]:
sheet_name = to_process[3]
print (sheet_name)
df = pd.read_excel(input_file,sheet_name = sheet_name)
df.head(10)

Health_Exchange


,Payer/PBM,# of Plans,Vijoice,Fyarro,Carvykti,Tecartus,Enhertu,Exkivity,Pluvicto,Cabometyx,...,Ayvakit,Tukysa,Qinlock,Bavencio,Blincyto,Cosela,Jelmyto,Polivy,Rozlytrek,Turalio
0,Centene Corporation,33 plans,82% Covered (PA/ST),71% Not Covered,71% Not Covered,77% Covered (PA/ST),71% Covered (PA/ST),57% Covered (PA/ST),98% Not Covered,97% Covered (PA/ST),...,82% Covered (PA/ST),100% Covered (PA/ST),82% Covered (PA/ST),67% Covered (PA/ST),100% Not Covered,100% Not Covered,71% Covered (PA/ST),66% Covered (PA/ST),82% Covered (PA/ST),81% Covered (PA/ST)
1,Florida Blue,5 plans,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Preferred,100% Not Covered,86% Covered (PA/ST),...,86% Covered (PA/ST),86% Covered (PA/ST),86% Covered (PA/ST),100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,86% Covered (PA/ST),86% Covered (PA/ST)
2,"Kaiser Foundation Health Plans, Inc.",11 plans,90% Covered (PA/ST),90% Covered (PA/ST),100% Covered (PA/ST),73% Covered (PA/ST),76% Covered,81% Covered (PA/ST),100% Not Covered,86% Covered,...,82% Covered (PA/ST),86% Covered,77% Covered (PA/ST),73% Covered (PA/ST),76% Covered,73% Covered (PA/ST),85% Not Covered,73% Covered (PA/ST),86% Covered,81% Covered (PA/ST)
3,Bright Health,16 plans,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,...,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered
4,Oscar Insurance,34 plans,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,93% Not Covered,...,100% Not Covered,69% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered
5,Health Care Service Corporation,7 plans,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Covered (PA/ST),100% Not Covered,100% Covered (PA/ST),...,100% Covered (PA/ST),97% Covered (PA/ST),100% Covered (PA/ST),100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Covered (PA/ST),83% Covered (PA/ST)
6,"Anthem, Inc.",20 plans,100% Covered (PA/ST),100% Covered (PA/ST),100% Not Covered,100% Not Covered,100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),...,100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST)
7,Blue Shield of California,3 plans,100% Covered,100% Not Covered,100% Covered,100% Not Covered,100% Not Covered,100% Covered (PA/ST),100% Not Covered,100% Covered (PA/ST),...,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered
8,"Molina Health Care, Inc.",16 plans,68% Covered (PA/ST),68% Covered (PA/ST),68% Covered (PA/ST),93% Covered (PA/ST),93% Covered (PA/ST),93% Covered (PA/ST),100% Not Covered,100% Covered (PA/ST),...,93% Covered (PA/ST),93% Covered (PA/ST),93% Covered (PA/ST),93% Covered (PA/ST),93% Covered (PA/ST),93% Covered (PA/ST),93% Covered (PA/ST),93% Covered (PA/ST),93% Covered (PA/ST),93% Covered (PA/ST)
9,Blue Cross Blue Shield of North Carolina,2 plans,100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Not Covered,100% Not Covered,100% Covered (PA/ST),100% Not Covered,100% Covered (PA/ST),...,100% Covered (PA/ST),100% Covered (PA/ST),100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Not Covered,100% Covered (PA/ST),100% Covered (PA/ST)


In [21]:
df_health_exchange_p = clean_percentage_data(df) # Health_Exchange
df_health_exchange_p

Trade Name,Centene Corporation,Florida Blue,"Kaiser Foundation Health Plans, Inc.",Bright Health,Oscar Insurance,Health Care Service Corporation,"Anthem, Inc.",Blue Shield of California,"Molina Health Care, Inc.",Blue Cross Blue Shield of North Carolina,...,McLaren Health Care Corporation,Capital Health Plan,Renown Health,Chinese Community Health Plan Corporation,"Community Care, Inc.",Community Health Network of Washington,Wisconsin Physicians Service Insurance Corporation,Cox Health,"Paramount Health Care, Inc.",Denver Health Medical Plan
Vijoice,82.0,100.0,90.0,100.0,100.0,100.0,100.0,100.0,68.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
Fyarro,71.0,100.0,90.0,100.0,100.0,100.0,100.0,100.0,68.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
Carvykti,71.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,68.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
Tecartus,77.0,100.0,73.0,100.0,100.0,100.0,100.0,100.0,93.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
Enhertu,71.0,100.0,76.0,100.0,100.0,100.0,100.0,100.0,93.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Cosela,100.0,100.0,73.0,100.0,100.0,100.0,100.0,100.0,93.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
Jelmyto,71.0,100.0,85.0,100.0,100.0,100.0,100.0,100.0,93.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
Polivy,66.0,100.0,73.0,100.0,100.0,100.0,100.0,100.0,93.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
Rozlytrek,82.0,86.0,86.0,100.0,100.0,100.0,100.0,100.0,93.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0


In [22]:
df_health_exchange_c = clean_coverage_data(df) # Health_Exchange
df_health_exchange_c

Trade Name,Centene Corporation,Florida Blue,"Kaiser Foundation Health Plans, Inc.",Bright Health,Oscar Insurance,Health Care Service Corporation,"Anthem, Inc.",Blue Shield of California,"Molina Health Care, Inc.",Blue Cross Blue Shield of North Carolina,...,Capital Health Plan,Renown Health,Chinese Community Health Plan Corporation,"Community Care, Inc.",Community Health Network of Washington,Wisconsin Physicians Service Insurance Corporation,Cox Health,"Paramount Health Care, Inc.",Denver Health Medical Plan,Ascension
Vijoice,Covered (PA/ST),Not Covered,Covered (PA/ST),Not Covered,Not Covered,Not Covered,Covered (PA/ST),Covered,Covered (PA/ST),Covered (PA/ST),...,Not Covered,Covered (PA/ST),Not Covered,Not Covered,Covered (PA/ST),Covered (PA/ST),Preferred (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST)
Fyarro,Not Covered,Not Covered,Covered (PA/ST),Not Covered,Not Covered,Not Covered,Covered (PA/ST),Not Covered,Covered (PA/ST),Covered (PA/ST),...,Not Covered,Covered (PA/ST),Not Covered,Not Covered,Covered (PA/ST),Covered (PA/ST),Not Covered,Covered (PA/ST),Not Covered,Covered (PA/ST)
Carvykti,Not Covered,Not Covered,Covered (PA/ST),Not Covered,Not Covered,Not Covered,Not Covered,Covered,Covered (PA/ST),Covered (PA/ST),...,Not Covered,Covered (PA/ST),Not Covered,Not Covered,Covered (PA/ST),Not Covered,Not Covered,Covered (PA/ST),Not Covered,Covered
Tecartus,Covered (PA/ST),Not Covered,Covered (PA/ST),Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Covered (PA/ST),Not Covered,...,Not Covered,Covered (PA/ST),Not Covered,Not Covered,Covered (PA/ST),Covered (PA/ST),Not Covered,Covered,Covered,Not Covered
Enhertu,Covered (PA/ST),Not Covered,Covered,Not Covered,Not Covered,Not Covered,Covered (PA/ST),Not Covered,Covered (PA/ST),Not Covered,...,Not Covered,Covered (PA/ST),Not Covered,Not Covered,Covered (PA/ST),Covered (PA/ST),Not Covered,Covered,Covered,Not Covered
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Cosela,Not Covered,Not Covered,Covered (PA/ST),Not Covered,Not Covered,Not Covered,Covered (PA/ST),Not Covered,Covered (PA/ST),Not Covered,...,Not Covered,Not Covered,Not Covered,Not Covered,Covered (PA/ST),Not Covered,Covered,Covered,Covered,Not Covered
Jelmyto,Covered (PA/ST),Not Covered,Not Covered,Not Covered,Not Covered,Not Covered,Covered (PA/ST),Not Covered,Covered (PA/ST),Not Covered,...,Not Covered,Covered (PA/ST),Not Covered,Not Covered,Covered (PA/ST),Covered (PA/ST),Not Covered,Covered,Covered (PA/ST),Not Covered
Polivy,Covered (PA/ST),Not Covered,Covered (PA/ST),Not Covered,Not Covered,Not Covered,Covered (PA/ST),Not Covered,Covered (PA/ST),Not Covered,...,Not Covered,Covered (PA/ST),Not Covered,Not Covered,Covered (PA/ST),Covered (PA/ST),Not Covered,Covered,Covered,Not Covered
Rozlytrek,Covered (PA/ST),Covered (PA/ST),Covered,Not Covered,Not Covered,Covered (PA/ST),Covered (PA/ST),Not Covered,Covered (PA/ST),Covered (PA/ST),...,Covered (PA/ST),Covered (PA/ST),Not Covered,Not Covered,Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered,Covered (PA/ST),Covered (PA/ST)


In [23]:
df_health_exchange_num = get_plans(df) # Health_Exchange
df_health_exchange_num

,Payer/PBM,# of Plans
0,Centene Corporation,33
1,Florida Blue,5
2,"Kaiser Foundation Health Plans, Inc.",11
3,Bright Health,16
4,Oscar Insurance,34
...,...,...
104,Wisconsin Physicians Service Insurance Corpora...,3
105,Cox Health,1
106,"Paramount Health Care, Inc.",1
107,Denver Health Medical Plan,2


## Managed medicaid

In [24]:
sheet_name = to_process[4]
print (sheet_name)
df = pd.read_excel(input_file,sheet_name = sheet_name)
df.head(10)

Managed medicaid


,Payer/PBM,# of Plans,Vijoice,Fyarro,Carvykti,Tecartus,Enhertu,Exkivity,Pluvicto,Cabometyx,...,Ayvakit,Tukysa,Qinlock,Bavencio,Blincyto,Cosela,Jelmyto,Polivy,Rozlytrek,Turalio
0,State of California,24 plans,100% Covered (PA/ST),100% Preferred,100% Covered (PA/ST),100% Covered (PA/ST),100% Preferred,100% Preferred,100% Not Covered,100% Preferred,...,100% Preferred,100% Covered (PA/ST),82% Covered (PA/ST),100% Preferred,100% Preferred,100% Covered (PA/ST),100% Preferred,100% Preferred,100% Preferred,100% Preferred
1,Centene Corporation,24 plans,56% Covered (PA/ST),59% Covered (PA/ST),78% Covered (PA/ST),58% Covered (PA/ST),70% Covered (PA/ST),64% Covered (PA/ST),88% Covered (PA/ST),69% Covered (PA/ST),...,68% Covered (PA/ST),70% Covered (PA/ST),86% Covered (PA/ST),70% Covered (PA/ST),70% Covered (PA/ST),88% Covered (PA/ST),70% Covered (PA/ST),69% Covered (PA/ST),70% Covered (PA/ST),73% Covered (PA/ST)
2,State of Texas,44 plans,98% Covered,100% Not Covered,100% Not Covered,98% Covered,98% Covered,100% Preferred,100% Not Covered,100% Preferred,...,100% Preferred,100% Preferred,77% Covered (PA/ST),98% Covered,98% Covered,82% Covered,72% Covered,98% Covered,100% Preferred,98% Covered
3,State of Florida,12 plans,100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered,100% Covered,100% Covered (PA/ST),100% Not Covered,100% Covered (PA/ST),...,100% Covered,100% Covered,100% Not Covered,100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered,100% Covered
4,"Anthem, Inc.",16 plans,94% Covered (PA/ST),100% Covered (PA/ST),71% Covered (PA/ST),71% Covered (PA/ST),94% Covered (PA/ST),94% Covered (PA/ST),94% Covered (PA/ST),94% Covered (PA/ST),...,94% Covered (PA/ST),94% Covered (PA/ST),100% Not Covered,94% Covered (PA/ST),94% Covered (PA/ST),66% Preferred (PA/ST),94% Covered (PA/ST),94% Covered (PA/ST),94% Covered (PA/ST),94% Covered (PA/ST)
5,State of Pennsylvania,10 plans,100% Covered (PA/ST),79% Covered (PA/ST),79% Covered (PA/ST),67% Covered (PA/ST),100% Covered (PA/ST),79% Covered (PA/ST),100% Covered (PA/ST),100% Preferred (PA/ST),...,100% Preferred (PA/ST),100% Preferred (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),100% Covered (PA/ST),67% Covered (PA/ST),67% Covered (PA/ST),100% Covered (PA/ST),100% Preferred (PA/ST),100% Preferred (PA/ST)
6,"UnitedHealth Group, Inc.",23 plans,81% Covered (PA/ST),72% Not Covered,73% Not Covered,81% Not Covered,84% Not Covered,90% Covered (PA/ST),81% Covered (PA/ST),84% Preferred (PA/ST),...,81% Covered (PA/ST),81% Covered (PA/ST),100% Covered (PA/ST),84% Not Covered,84% Not Covered,81% Covered (PA/ST),83% Not Covered,73% Not Covered,78% Preferred (PA/ST),78% Preferred (PA/ST)
7,State of Ohio,5 plans,100% Covered (PA/ST),87% Covered (PA/ST),87% Covered (PA/ST),87% Covered (PA/ST),49% Preferred (PA/ST),100% Covered (PA/ST),51% Not Covered,87% Covered (PA/ST),...,100% Covered (PA/ST),51% Covered (PA/ST),100% Not Covered,87% Covered (PA/ST),87% Covered (PA/ST),100% Covered (PA/ST),49% Preferred (PA/ST),87% Covered (PA/ST),62% Preferred (PA/ST),62% Preferred (PA/ST)
8,"Molina Health Care, Inc.",15 plans,57% Covered (PA/ST),58% Covered (PA/ST),63% Covered (PA/ST),61% Not Covered,47% Covered (PA/ST),47% Covered (PA/ST),53% Not Covered,47% Covered (PA/ST),...,47% Covered (PA/ST),47% Covered (PA/ST),93% Covered (PA/ST),47% Covered (PA/ST),47% Covered (PA/ST),47% Covered (PA/ST),47% Covered (PA/ST),47% Covered (PA/ST),49% Preferred (PA/ST),49% Preferred (PA/ST)
9,State of North Carolina,6 plans,100% Covered,62% Covered,83% Not Covered,83% Not Covered,100% Covered,100% Covered,62% Not Covered,62% Covered (PA/ST),...,62% Covered (PA/ST),100% Covered,100% Not Covered,100% Covered,79% Covered,62% Covered,100% Covered,100% Covered,100% Covered,100% Covered


In [25]:
df_managed_medicaid_p = clean_percentage_data(df) # Managed medicaid
df_managed_medicaid_p

Trade Name,State of California,Centene Corporation,State of Texas,State of Florida,"Anthem, Inc.",State of Pennsylvania,"UnitedHealth Group, Inc.",State of Ohio,"Molina Health Care, Inc.",State of North Carolina,...,"Western Oregon Advanced Health, LLC.",Merle West Medical Center,Geisinger Health System Corporation,"Humana, Inc.",ITASCA Medical Care,"Amida Care, Inc.",Health Partners Plans,Visiting Nurse Service of New York,Medica Health Plans,Providence Health System
Vijoice,100.0,56.0,98.0,100.0,94.0,100.0,81.0,100.0,57.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
Fyarro,100.0,59.0,100.0,100.0,100.0,79.0,72.0,87.0,58.0,62.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
Carvykti,100.0,78.0,100.0,100.0,71.0,79.0,73.0,87.0,63.0,83.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
Tecartus,100.0,58.0,98.0,100.0,71.0,67.0,81.0,87.0,61.0,83.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
Enhertu,100.0,70.0,98.0,100.0,94.0,100.0,84.0,49.0,47.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Cosela,100.0,88.0,82.0,100.0,66.0,67.0,81.0,100.0,47.0,62.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
Jelmyto,100.0,70.0,72.0,100.0,94.0,67.0,83.0,49.0,47.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
Polivy,100.0,69.0,98.0,100.0,94.0,100.0,73.0,87.0,47.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
Rozlytrek,100.0,70.0,100.0,100.0,94.0,100.0,78.0,62.0,49.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0


In [26]:
df_managed_medicaid_c = clean_coverage_data(df) # Managed medicaid
df_managed_medicaid_c

Trade Name,State of California,Centene Corporation,State of Texas,State of Florida,"Anthem, Inc.",State of Pennsylvania,"UnitedHealth Group, Inc.",State of Ohio,"Molina Health Care, Inc.",State of North Carolina,...,"Western Oregon Advanced Health, LLC.",Merle West Medical Center,Geisinger Health System Corporation,"Humana, Inc.",ITASCA Medical Care,"Amida Care, Inc.",Health Partners Plans,Visiting Nurse Service of New York,Medica Health Plans,Providence Health System
Vijoice,Covered (PA/ST),Covered (PA/ST),Covered,Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered,...,Covered (PA/ST),Covered (PA/ST),Preferred (PA/ST),Covered (PA/ST),Not Covered,Not Covered,Covered (PA/ST),Covered (PA/ST),Covered,Covered (PA/ST)
Fyarro,Preferred,Covered (PA/ST),Not Covered,Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Not Covered,Covered (PA/ST),Covered (PA/ST),Covered,...,Not Covered,Covered (PA/ST),Preferred (PA/ST),Covered (PA/ST),Not Covered,Not Covered,Covered (PA/ST),Covered (PA/ST),Not Covered,Not Covered
Carvykti,Covered (PA/ST),Covered (PA/ST),Not Covered,Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Not Covered,Covered (PA/ST),Covered (PA/ST),Not Covered,...,Covered (PA/ST),Covered (PA/ST),Preferred (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST)
Tecartus,Covered (PA/ST),Covered (PA/ST),Covered,Covered,Covered (PA/ST),Covered (PA/ST),Not Covered,Covered (PA/ST),Not Covered,Not Covered,...,Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered,Covered (PA/ST),Covered (PA/ST)
Enhertu,Preferred,Covered (PA/ST),Covered,Covered,Covered (PA/ST),Covered (PA/ST),Not Covered,Preferred (PA/ST),Covered (PA/ST),Covered,...,Covered (PA/ST),Covered (PA/ST),Preferred (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered,Covered (PA/ST),Covered (PA/ST)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Cosela,Covered (PA/ST),Covered (PA/ST),Covered,Covered (PA/ST),Preferred (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered,...,Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered,Covered (PA/ST),Covered (PA/ST)
Jelmyto,Preferred,Covered (PA/ST),Covered,Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Not Covered,Preferred (PA/ST),Covered (PA/ST),Covered,...,Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered,Covered (PA/ST),Covered (PA/ST)
Polivy,Preferred,Covered (PA/ST),Covered,Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Not Covered,Covered (PA/ST),Covered (PA/ST),Covered,...,Covered (PA/ST),Covered (PA/ST),Preferred (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered (PA/ST),Covered,Covered (PA/ST),Covered (PA/ST)
Rozlytrek,Preferred,Covered (PA/ST),Preferred,Covered,Covered (PA/ST),Preferred (PA/ST),Preferred (PA/ST),Preferred (PA/ST),Preferred (PA/ST),Covered,...,Covered (PA/ST),Covered (PA/ST),Preferred (PA/ST),Preferred (PA/ST),Covered (PA/ST),Preferred (PA/ST),Preferred (PA/ST),Preferred (PA/ST),Covered,Preferred (PA/ST)


In [27]:
df_managed_medicaid_num = get_plans(df) # Managed medicaid
df_managed_medicaid_num

,Payer/PBM,# of Plans
0,State of California,24
1,Centene Corporation,24
2,State of Texas,44
3,State of Florida,12
4,"Anthem, Inc.",16
...,...,...
88,"Amida Care, Inc.",1
89,Health Partners Plans,1
90,Visiting Nurse Service of New York,1
91,Medica Health Plans,2


In [28]:
writer = pd.ExcelWriter('output.xlsx')

melt(df_commercial_p).to_excel(writer, 'commercial %')
melt(df_commercial_c).to_excel(writer, 'commercial_coverage')
df_commercial_num.to_excel(writer, 'commercial_plans_list')

melt(df_medicare_p).to_excel(writer, 'medicare %')
melt(df_medicare_c).to_excel(writer, 'medicare_coverage')
df_medicare_num.to_excel(writer, 'medicare_plans_list')

melt(df_state_medicaid_p).to_excel(writer, 'state_medicaid %')
melt(df_state_medicaid_c).to_excel(writer, 'state_medicaid_coverage')
df_state_medicaid_num.to_excel(writer, 'state_medicaid_plans_list')

melt(df_health_exchange_p).to_excel(writer, 'health_exchange %')
melt(df_health_exchange_c).to_excel(writer, 'health_exchange_coverage')
df_health_exchange_num.to_excel(writer, 'health_exchange_plans_list')

melt(df_managed_medicaid_p).to_excel(writer, 'managed_medicaid %')
melt(df_managed_medicaid_c).to_excel(writer, 'managed_medicaid_coverage')
df_managed_medicaid_num.to_excel(writer, 'managed_medicaid_plans_list')

writer.save()